To Do:
- create downloadable csv for each jurisdiction
- deal with duble field entries esp from the simple normalizing

In [1]:
jurs = [
        "MI",
        "UT",
        "ME"
        ]

In [2]:
import pandas as pd
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import PyPDF2

import pikepdf

import os
from os import walk
import os.path
from os import path
import numpy as np
#!pip install py-readability-metrics
from readability import Readability
#!python -m nltk.downloader punkt

import time
from datetime import date

import spacy
from sklearn.preprocessing import normalize

from joblib import dump, load


import networkx as nx

import math

from sklearn.metrics.pairwise import cosine_similarity# define matrix with all zero values

#import nltk
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
def vectorize(text):
    output = nlp(str(text)).vector   
    return output

def norm(row):
    try:
        matrix = row.reshape(1,-1).astype(np.float64)
        return normalize(matrix, axis=1, norm='l1')[0]
    except Exception as e: 
        print("===================")
        print(row)
        print("-------------------")
        print(e)
        print("===================")
        #return np.NaN

In [20]:
def regex_norm_field(text):
    # Takes an auto-generated form field name and uses
    # regex to convert it into an Assembly Line standard field.
    # See https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables/

    regex_list = [

        # Personal info
        ## Name & Bio
        ["^((My|Full( legal)?) )?Name$","users1_name"],
        ["^(Typed or )?Printed Name\s?\d*$","users1_name"],
        ["^(DOB|Date of Birth|Birthday)$","users1_birthdate"],
        ## Address
        ["^(Street )?Address$","users1_address_line_one"],
        ["^City State Zip$","users1_address_line_two"],
        ["^City$","users1_address_city"],
        ["^State$","users1_address_state"],
        ["^Zip( Code)?$","users1_address_zip"],
        ## Contact
        ["^(Phone|Telephone)$","users1_phone_number"],
        ["^Email( Adress)$","users1_email"],
        
        # Parties
        ["plaintiff","plantiff1_name"],
        ["defendant","defendant1_name"],
        ["petitioners","petitioners1_name"],
        ["respondents","respondents1_name"],

        # Court info
        ["^(Court\s)?Case\s?(No|Number)?\s?A?$","docket_number"],
        ["^File\s?(No|Number)?\s?A?$","docket_number"],

        # Form info
        ["^(Signature|Sign( here)?)\s?\d*$","users1_signature"],
        ["^Date\s?\d*$","signature_date"],
    ]

    for regex in regex_list:
        text = re.sub(regex[0],regex[1],text, flags=re.IGNORECASE)
    return text

included_fields = load('../data/processed/ML/norm_fields/included_fields.joblib') 
jurisdictions = load('../data/processed/ML/norm_fields/jurisdictions.joblib') 
groups = load('../data/processed/ML/norm_fields/groups.joblib') 
clf_field_names = load('../data/processed/ML/norm_fields/clf_field_names.joblib') 

stop_words = set(stopwords.words('english'))

def reformat_field(text,max_length=30):
    # h/t https://towardsdatascience.com/nlp-building-a-summariser-68e0c19e3a93
    
    #print(text)
    
    orig_title = text.lower()
    orig_title = re.sub("[^a-zA-Z]+"," ",orig_title)
    orig_title_words = orig_title.split()
   
    deduped_sentence = []
    for word in orig_title_words:
        if word not in deduped_sentence:
            deduped_sentence.append(word)
            
    filtered_sentence = [w for w in deduped_sentence if not w.lower() in stop_words]   

    filtered_title_words = filtered_sentence
    
    characters = len(' '.join(filtered_title_words))
    
    if characters > 0:

        words = len(filtered_title_words)
        av_word_len = math.ceil(len(' '.join(filtered_title_words))/len(filtered_title_words))
        x_words = math.floor((max_length)/av_word_len)


        sim_mat = np.zeros([len(filtered_title_words),len(filtered_title_words)])# will populate it with cosine_similarity values 
        # for each word compared to other
        for i in range(len(filtered_title_words)):
            for j in range(len(filtered_title_words)):
                if i != j:
                    sim_mat[i][j] = cosine_similarity(nlp(filtered_title_words[i]).vector.reshape(1,300), nlp(filtered_title_words[j]).vector.reshape(1,300))[0,0]

        try:
            nx_graph = nx.from_numpy_array(sim_mat)
            scores = nx.pagerank(nx_graph)# print final values of words
            sorted_scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

            if x_words > len(scores):
                x_words=len(scores)

            i = 0
            new_title = ""
            for x in filtered_title_words:
                #print(scores[i],sorted_scores[x_words][1])
                if scores[i] >= sorted_scores[x_words-1][1]: 
                    if len(new_title)>0: new_title+="_"
                    new_title += x
                i+=1

            return new_title
        except:
            return '_'.join(filtered_title_words)
    else:
        if re.search("^(\d+)$", text):
            return "unknown"
        else:
            return re.sub("\s+","_",text.lower())
    

def normalize_name(jur,group,n,per,last_field,this_field):
    
    # Add hard coded conversions maybe by calling a function
    # if returns 0 then fail over to ML or otherway around poor prob -> check hard-coded

    if this_field not in included_fields:
        this_field = reCase(this_field)

        out_put = regex_norm_field(this_field)
        conf = 1.0

        if out_put==this_field:
            params = []
            for item in jurisdictions:
                if jur== item:
                    params.append(1)
                else:
                    params.append(0)
            for item in groups:
                if group== item:
                    params.append(1)
                else:
                    params.append(0)
            params.append(n)
            params.append(per)
            for vec in norm(vectorize(this_field)):
                params.append(vec)
            #for vec in norm(vectorize(last_field)):
            #    params.append(vec)

            for item in included_fields:
                if last_field==item:
                    params.append(1)
                else:
                    params.append(0)

            pred = clf_field_names.predict([params])
            prob = clf_field_names.predict_proba([params])

            conf = prob[0].tolist()[prob[0].tolist().index(max(prob[0].tolist()))]
            out_put = pred[0]
            
    else:
        out_put = this_field
        conf = 1
            
    if out_put in included_fields:
        if conf >= 0:
            return "*"+out_put,conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
        else:
            return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf
    else:
        return reformat_field(this_field),conf #+"| was <i>%s</i> (%.2f conf)"%(this_field,conf) #, conf


In [21]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [22]:
def read_pdf (file):
    try:
        pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        if pdfFile.isEncrypted:
            try:
                pdfFile.decrypt('')
                #print ('File Decrypted (PyPDF2)')
            except:
                #
                #
                # This didn't go so well on my Windows box so I just ran this in the pdf folder's cmd:
                # for %f in (*.*) do copy %f temp.pdf /Y && "C:\Program Files (x86)\qpdf-8.0.2\bin\qpdf.exe" --password="" --decrypt temp.pdf %f
                #
                #
                #
                
                command="cp "+file+" tmp/temp.pdf; qpdf --password='' --decrypt tmp/temp.pdf "+file
                os.system(command)
                #print ('File Decrypted (qpdf)')
                #re-open the decrypted file
                pdfFile = PyPDF2.PdfFileReader(open(file, "rb"))
        text = ""
        for page in pdfFile.pages:
            text = text + " " + page.extractText()
        return text
    except:
        return ""


In [23]:
def parse_form(fileloc,title=None,jur=None,cat=None,normalize=1):
    f = PyPDF2.PdfFileReader(fileloc)
    
    if f.isEncrypted:
        pdf = pikepdf.open(fileloc, allow_overwriting_input=True)
        pdf.save(fileloc)
        f = PyPDF2.PdfFileReader(fileloc)
    
    npages = f.getNumPages()
    ff = f.getFields()
    if ff:
        fields = list(ff.keys())
    else:
        fields = []
    f_per_page = len(fields)/npages
    text = read_pdf(fileloc)
    try:
        readbility = int(Readability(text).flesch_kincaid().grade_level)
    except:
        readbility = None
    
    if title is None:
        title = reCase(re.search("(.*)\n",text).group(1).strip())
        
    if normalize==1:
        i = 0 
        length = len(fields)
        last = "null"
        new_fields = []
        new_fields_conf = []
        for field in fields:
            #print(jur,cat,i,i/length,last,field)
            this_field,this_conf = normalize_name(jur,cat,i,i/length,last,field)
            new_fields.append(this_field)
            new_fields_conf.append(this_conf)
            last = field
        
        new_fields = [v + "__" + str(new_fields[:i].count(v) + 1) if new_fields.count(v) > 1 else v for i, v in enumerate(new_fields)]
    else:
        new_fields = fields
    
    stats = {
            "title":title,
            "category":cat,
            "pages":npages,
            "reading grade level": readbility,
            "avg fields per page": f_per_page,
            "fields":new_fields,
            "fields_conf":new_fields_conf,
            "fields_old":fields
            }    
    
    my_pdf = pikepdf.Pdf.open(fileloc)
    fields_too = my_pdf.Root.AcroForm.Fields
    k =0
    for field in new_fields:
        fields_too[k].T = re.sub("^\*","",field)
        k+=1
        
    #f2.T = 'new_hospital_name'
    filename = re.search("\/(\w*\.pdf)$",fileloc).groups()[0]
    my_pdf.save('../data/processed/forms/%s'%(filename))
    
    return stats

In [25]:
#parse_form("../data/processed/www.utcourts.gov/forms/898269a99ff1c65be10b1ae35bb34ba469fc14b7301b7ed7b126d195.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/www.utcourts.gov/forms/2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065cd27bde8c7.pdf",title=None,jur="UT",cat=None,normalize=1)
parse_form("../data/processed/www.utcourts.gov/forms/6ec7576210513907e699b5adf3397639507c688801a60bc34c201984.pdf",title=None,jur="UT",cat=None,normalize=1)
#parse_form("../data/processed/mjbportal.courts.maine.gov/forms/1519fe450d870a36a428a0b006c0665a.pdf",title=None,jur="UT",cat=None,normalize=1)

{'title': '',
 'category': None,
 'pages': 2,
 'reading grade level': 14,
 'avg fields per page': 16.5,
 'fields': ['*users1_name__1',
  '*users1_address_line_one',
  '*users1_address_line_two',
  '*users1_phone_number',
  'email',
  'undefined__1',
  'plaintiff_petitioner',
  '*docket_number',
  'judge',
  'defendant_respondent',
  'commissioner_domestic_cases',
  'city_state_country',
  'undefined__2',
  '*users1_signature__1',
  '*users1_name__2',
  'service_members_limited__1',
  'board_district_court_judges__1',
  'page__1',
  'persons_name_row__1',
  'service_mail_person__1',
  'service_person_charge__1',
  'persons_name_row__2',
  'service_mail_person__2',
  'service_person_charge__2',
  'persons_name_row__3',
  'service_mail_person__3',
  'service_person_charge__3',
  '*signature_date',
  '*users1_signature__2',
  '*users1_name__3',
  'service_members_limited__2',
  'board_district_court_judges__2',
  'page__2'],
 'fields_conf': [1.0,
  1.0,
  1.0,
  1.0,
  0.61,
  0.54,
  1.0,

In [26]:
files_df_ut = pd.read_csv("../data/raw/www.utcourts.gov/form_data.csv")
files_df_mi = pd.read_csv("../data/raw/www.courts.michigan.gov/form_data.csv")
files_df_me = pd.read_csv("../data/raw/mjbportal.courts.maine.gov/form_data.csv")

df = pd.concat([files_df_ut,files_df_mi,files_df_me],ignore_index=True)
df.head()

,id,jurisdiction,source,title,group,url,filename,downloaded
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11


In [27]:
df["pages"] = ""
df["fields"] = ""
df["fields_conf"] = ""
df["fields_old"] = ""
df["f_per_p"] = ""
df["reading"] = ""

for index,row in df.iterrows():
    if (row["pages"] == "") & (row["jurisdiction"] in jurs):
        try:
            stats = parse_form("../data/processed/"+row["source"]+"/forms/"+row["id"]+".pdf",row["title"],row["jurisdiction"],row["group"],1)
            df.at[index, 'pages'] = stats["pages"]
            df.at[index, 'fields'] = stats["fields"]
            df.at[index, 'fields_conf'] = stats["fields_conf"]
            df.at[index, 'fields_old'] = stats["fields_old"]
            df.at[index, 'f_per_p'] = stats["avg fields per page"]
            df.at[index, 'reading'] = stats["reading grade level"]
            #print(index)
        except:
            print("error: "+"../data/raw/"+row["source"]+"/forms/"+row["id"]+".pdf")
        
df

error: ../data/raw/www.utcourts.gov/forms/e6124de37c9e4ef524d46b368c582ac2adf999778566d68b1a3dd820.pdf
error: ../data/raw/www.utcourts.gov/forms/032c7267803815f129aae6465e8536fcc80dedb75360cf50dff3baeb.pdf


error: ../data/raw/www.courts.michigan.gov/forms/531b8bf75814ce2a9757fcae1b1926bacce15ab816f721165bf22b85.pdf


error: ../data/raw/www.courts.michigan.gov/forms/bfa644ad53e07138e85ce99e4eb01ed784805b9bd6fd08844623a4e8.pdf


error: ../data/raw/www.courts.michigan.gov/forms/6b79c1e8fda21d9a395e262f564c03a15af2761add0e4273f879d477.pdf
error: ../data/raw/www.courts.michigan.gov/forms/f9b72d647368a16b4abd3ffe22c2c7fa0204d4cdd1d1f19a110f00a1.pdf


error: ../data/raw/www.courts.michigan.gov/forms/bba53d81e3ffa8d9769cacb36b726b1c5c4888aba26576cb5fd16c31.pdf


...

error: ../data/raw/mjbportal.courts.maine.gov/forms/2207f5f02124e76f982bc5d165d273a2.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/c59d05adba00ca2697a8b0b2d99e4f97.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/2ee94cacfbfcb947373b0ce489b91d21.pdf


error: ../data/raw/mjbportal.courts.maine.gov/forms/a4df4c1d316c399b6a8d6f231e3a50a5.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/3a5f093d3ca3f28b058c697700bc30d9.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/1b6b548650095e930699f5d08945665f.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/b57fc403d5a1d59f0e423f7afa035ea0.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/b3828053373ab99c63e110a8d0eb43ce.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/711e6a72de9b1f977d4920c4ca20c44f.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/fa0d535fbf5d8fdff19e0ae97d125e16.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/ad5c2162fd247f773983ed441e2fec8f.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/8433c0560dee8c1d6742487ffdf7d6b5.pdf


error: ../data/raw/mjbportal.courts.maine.gov/forms/c4e486de77c29131978087b0d874591d.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/1249d51795e8c5facf0c2ce02f4ff515.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/c2f027d39998194a2000508be193d6da.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/abae2b5a2f3555090ebbd6afb2b15f42.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/26a8756da9646ce02c990fdaba055b77.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/0c4b786a69a923a6955a05b5bd6fc846.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/8446022e1b6a747dfe8ef801d6695750.pdf


error: ../data/raw/mjbportal.courts.maine.gov/forms/219a85df049f2c58feba1c0aae0e0767.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/313a5fad0b1b79d39eae4c67f1add376.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/bf868f0a503e378a80d7a0b6ead2c01a.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/00190b06452535f7286e9009cb04072f.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/626de8ceb09625999bd43d76426356ea.pdf
error: ../data/raw/mjbportal.courts.maine.gov/forms/6ef6d30dd5c36ff5a0ddfc15ac0e8e4e.pdf


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,fields_conf,fields_old,f_per_p,reading
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2,"[name__1, name__2, *docket_number, *users1_add...","[0.61, 0.65, 1.0, 1.0, 1.0, 0.61, 0.53, 0.57, ...","[NAME 1, NAME 2, CASE NUMBER, ADDRESS, DATE OF...",7.5,10
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2,"[*docket_number, *users1_birthdate, male, juve...","[1.0, 1.0, 0.7, 0.61, 1.0, 1.0, 1.0, 1.0, 0.71...","[Court Case, Date of Birth, Male, Juveniles Na...",19.0,11
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5,"[probation_department_court, special_condition...","[0.73, 0.65, 0.7, 0.54, 0.68, 0.67, 0.62, 0.59...",[of the probation department or by order of th...,4.8,15
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","[Name, Address, City State Zip, Phone, Email, ...",14.666667,10
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1,"[*users1_name, *users1_address_line_one, *user...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","[Name, Address, City State Zip, Phone, Email, ...",19.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1741,7db3ccf8291cc25aadee526d17862ac0,ME,mjbportal.courts.maine.gov,Notification That Observation is No Longer Nec...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CR-JV-253.pdf,2021-11-16,1,"[superior_court, unified_criminal_docket, dist...","[0.65, 0.62, 0.59, 0.57, 0.54, 0.72, 0.62, 0.5...","[Superior Court, Unified Criminal Docket, Dist...",11.0,15
1742,baec88eb3268a0754d3c1eb949acc1fe,ME,mjbportal.courts.maine.gov,Civil Summary Sheet,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-001.pdf,2021-11-16,4,"[court_appropriate_box, superior_court_county,...","[0.6, 0.63, 0.63, 0.54, 0.58, 0.66, 0.62, 0.65...",[COUNTY OF FILING OR DISTRICT COURT JURISDICTI...,40.5,14
1743,5ccfa502e8a951986add0248ea9a2bef,ME,mjbportal.courts.maine.gov,Complaint for Residential Forcible Entry and D...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-007.pdf,2021-11-16,2,"[plaintiff__1, location_town, v, docket, defen...","[1.0, 0.72, 0.52, 0.63, 1.0, 0.69, 1.0, 1.0, 0...","[Plaintiff, Location Town, V, Docket No, Defen...",28.0,12
1744,aeb7825a71b2327c6b61676e83042fac,ME,mjbportal.courts.maine.gov,Subpoena on Motion for Contempt (*Self-represe...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-037.pdf,2021-11-16,2,"[plaintiff__1, superior_court, district_court,...","[1.0, 0.65, 0.59, 0.57, 0.72, 1.0, 0.63, 0.7, ...","[Plaintiff, Superior Court, District Court, Co...",12.0,11


In [ ]:
#df1 = pd.read_csv("../data/processed/form_data.csv")
#df1

In [ ]:
#df2 = df[(df["jurisdiction"]=="ME") & (df["pages"]!="")]
#df2

In [ ]:
#dfall = pd.concat([df1,df2],ignore_index=True)
#dfall

In [28]:
#dfall.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    
df.to_csv("../data/processed/form_data.csv", index=False, encoding="utf-8")    

In [ ]:
#os.system('cp ../data/processed/www.utcourts.gov/forms/* ../data/processed/forms/')
#os.system('cp ../data/processed/www.courts.michigan.gov/forms/* ../data/processed/forms/')